# Chapter 7. Parallel Processing

With parallel processing by using multiple cores, you can increase the amount of calculations your program can do in a given time frame without needing a faster processor. The main idea is to divide a problem into independent subunits and use multiple cores to solve those subunits in parallel.

Parallel processing is necessary to tackle large-scale problems. Companies produce massive quantities of data every day that need to be stored in multiple computers and analyzed. Scientists and engineers run parallel code on supercomputers to simulate massive systems.

Parallel processing allows you to take advantage of multicore CPUs as well as GPUs that work extremely well with highly parallel problems

## 7.1 Introduction to parallel processing

In order to parallelize a program, it is necessary to divide the problem into subunits that can "run independently (or almost independently) from each other

A problem where the subunits are totally independent from each other is called embarrassingly parallel. An element-wise operation on an array is a typical example--the operation needs to only know the element it is handling at the moment.Embarrassingly parallel problems are very easy to implement and perform very well on parallel architectures

Other problems may be divided into subunits but have to share some data to perform their calculations. In those cases, the implementation is less straightforward and can lead to performance issues because of the communication costs

Communication between processes is costly and can seriously hinder the performance of parallel programs. There exist two main ways to handle data communication in parallel programs:
1. **Shared memory**
2. **Distributed memory**

In shared memory, the subunits have access to the same memory space. The advantage of this approach is that you don't have to explicitly handle the communication as it is sufficient to write or read from the shared memory. However, problems arise when multiple processes try to access and change the same memory location at the same time. Care should be taken to avoid such conflict using synchronization techniques

In the distributed memory model, each process is completely separated from the others and possesses its own memory space. In this case, communication is handled explicitly between the processes. The communication overhead is typically costlier compared to shared memory as data can potentially travel through a network interface

Python can spawn and handle threads, but they can't be used to increase performance; due to the Python interpreter design, only one Python instruction is allowed to run at a time-- this mechanism is called **Global Interpreter Lock (GIL)**. What happens is that each time a thread executes a Python statement, the thread acquires a lock and, when the execution is completed, the same lock is released. Since the lock can be acquired only by one thread at a time, other threads are prevented from executing Python statements while some other thread holds the lock.

Even though the GIL prevents parallel execution of Python instructions, threads can still be used to provide concurrency in situations where the lock can be released, such as in timeconsuming I/O operations or in C extensions

The GIL can be completely sidestepped using processes instead of threads. Processes don't share the same memory area and are independent from each other--each process has its own interpreter. Processes have a few disadvantages: starting up a new process is generally slower than starting a new thread, they consume more memory, and inter-process communication can be slow. On the other hand, processes are still very flexible, and they scale better as they can be distributed on multiple machines.

### 7.1.1 Graphic processing units

Graphic processing units are special processors designed for computer graphics applications. Those applications usually require processing the geometry of a 3D scene and output an array of pixel to the screen. The operations performed by GPUs involve array and matrix operations on floating point numbers

GPUs are designed to run this graphics-related operation very efficiently, and they achieve this by adopting a highly parallel architecture. Compared to a CPU, a GPU has many more
(thousands) of small processing units. GPUs are intended to produce data at about 60 frames per second, which is much slower than the typical response time of a CPU, which possesses higher clock speeds.

GPUs possess a very different architecture from a standard CPU and are specialized for computing floating point operations. Therefore, to compile programs for GPUs, it is necessary to utilize special programming platforms, such as CUDA and OpenCL.

**Compute Unified Device Architecture (CUDA)** is a proprietary NVIDIA technology. It provides an API that can be accessed from other languages. CUDA provides the NVCC tool that can be used to compile GPU programs written in a language similar to C (CUDA C) as well as numerous libraries that implement highly optimized mathematical routines

**OpenCL** is an open technology with the ability of writing parallel programs that can be compiled for a variety of target devices (CPUs and GPUs of several vendors) and is a good option for non-NVIDIA devices

GPU programming is tricky and only specific use cases benefit from the GPU architecture. Programmers need to be aware of the costs incurred in memory transfers to and from the main memory and how to implement algorithms to take advantage of the GPU architecture

GPUs are great at increasing the amount of operations you can perform per unit of time (also called throughput); however, they require more time to prepare the data for processing. In contrast, CPUs are much faster at producing an individual result from scratch (also called latency).

## 7.2 Using multiple processes

The standard multiprocessing module can be used to quickly parallelize simple tasks by spawning several processes, while avoiding the GIL problem. Its interface is easy to use and includes several utilities to handle task submission and synchronization.

### 7.2.1 The process and pool classes

In [1]:
import multiprocessing 
import time 

class Process(multiprocessing.Process):
    def __init__(self, id):
        super(Process, self).__init__()
        self.id = id
    def run(self):
        time.sleep(1)
        print("I'm the process with id: {}".format(self.id))

if __name__ == '__main__':
    p = Process(0)
    p.start()

if __name__ == '__main__':
    p = Process(0)
    p.start()
    p.join()

I'm the process with id: 0
I'm the process with id: 0


In [2]:
if __name__ == '__main__':
    processes = Process(1), Process(2), Process(3), Process(4)
    [p.start() for p in processes]

I'm the process with id: 1
I'm the process with id: 2
I'm the process with id: 3
I'm the process with id: 4


The multiprocessing module exposes a convenient interface that makes it easy to assign and distribute tasks to a set of processes that reside in the multiprocessing.Pool class.

The multiprocessing.Pool class spawns a set of processes--called workers--and lets us submit tasks through the apply/apply_async and map/map_async methods.

The Pool.map method applies a function to each element of a list and returns the list of results. Its usage is equivalent to the built-in (serial) map

In [ ]:
pool = multiprocessing.Pool()
pool = multiprocessing.Pool(processes=4)

In [ ]:
def square(x):
    return x * x

inputs = [0,1,2,3,4]
outputs = pool.map(square, inputs)

In [ ]:
outputs_async = pool.map_async(square, inputs)
outputs = outputs_async.get()

In [ ]:
results_async = [pool.apply_async(square, i) for i in range(100))]
results = [r.get() for r in results_async]

### 7.2.2 The executor interface

In [ ]:
from concurrent.futures import ProcessPoolExecutor
executor = ProcessPoolExecutor(max_workers=4)
fut = executor.submit(square, 2)

In [ ]:
result = executor.map(square, [0, 1, 2, 3, 4])
list(result)

In [ ]:
from concurrent.futures import wait, as_completed

fut1 = executor.submit(square, 2)
fut2 = executor.submit(square, 3)
wait([fut1, fut2])

# Then you can extract the results using fut1.result() and fut2.result()
results = as_completed([fut1, fut2])
list(results)
# Result:
# [4, 9]

### 7.2.3 Monte carlo approximation of pi

In [ ]:
hits/total = area_circle/area_square = pi/4
pi = 4 * hits/total

In [3]:
import random

samples = 1000000
hits = 0

for i in range(samples):
    x = random.uniform(-1.0, 1.0)
    y = random.uniform(-1.0, 1.0)
    if x**2 + y**2 <= 1:
        hits += 1
pi = 4.0 * hits/samples

In [4]:
import multiprocessing
def sample():
    x = random.uniform(-1.0, 1.0)
    y = random.uniform(-1.0, 1.0)
    if x**2 + y**2 <= 1:
        return 1
    else:
        return 0
pool = multiprocessing.Pool()
results_async = [pool.apply_async(sample) for i in range(samples)]
hits = sum(r.get() for r in results_async)

In [6]:
!time python -c 'import pi; pi.pi_serial()'


real	0m5.820s
user	0m5.747s
sys	0m0.030s


In [7]:
!time python -c 'import pi; pi.pi_apply_async()'


real	8m35.980s
user	10m52.730s
sys	5m38.677s


In [ ]:
def sample_multiple(samples_partial):
    return sum(sample() for i in range(samples_partial))
n_tasks = 10
chunk_size = samples/n_tasks
pool = multiprocessing.Pool()
results_async = [pool.apply_async(sample_multiple, chunk_size)
                 for i in range(n_tasks)]
hits = sum(r.get() for r in results_async)

In [10]:
!time python -c 'import pi; pi.pi_apply_async_chunked()'

multiprocessing.pool.RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/boyuan/anaconda3/envs/Python/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/Users/boyuan/Desktop/OneDrive/Data science/Python/Python high performance 2nd/Python-High-Performance-Second-Edition-master/Chapter07/pi.py", line 38, in sample_multiple
    return sum(sample() for i in range(samples_partial))
TypeError: 'float' object cannot be interpreted as an integer
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/boyuan/Desktop/OneDrive/Data science/Python/Python high performance 2nd/Python-High-Performance-Second-Edition-master/Chapter07/pi.py", line 46, in pi_apply_async_chunked
    hits = sum(r.get() for r in results_async)
  File "/Users/boyuan/Desktop/OneDrive/Data science/Python/Python high performance 2nd/Python-Hig

### 7.2.4 Synchronization and locks

In [11]:
shared_variable = multiprocessing.Value('f')
shared_variable.value = 0

In [12]:
class Process(multiprocessing.Process):
    def __init__(self, counter):
        super(Process, self).__init__()
        self.counter = counter
    def run(self):
        for i in range(1000):
            self.counter.value += 1

In [13]:
def main():
    counter = multiprocessing.Value('i', lock=True)
    counter.value = 0
    processes = [Process(counter) for i in range(4)]
    [p.start() for p in processes]
    [p.join() for p in processes] # processes are done
    print(counter.value)

Since the lock can be acquired by only one process at a time, this method prevents multiple processes from executing the protected section of code at the same time.

In [ ]:
lock = multiprocessing.Lock()

class Process(multiprocessing.Process):
    def __init__(self, counter):
        super(Process, self).__init__()
        self.counter = counter
    def run(self):
        for i in range(1000):
            with lock: # acquire the lock
                 self.counter.value += 1
            # release the lock

## 7.3 Parallel Cython with OpenMP

In [ ]:
import numpy as np

def square_serial(double[:] inp):
    cdef int i, size
    cdef double[:] out
    size = inp.shape[0]
    out_np = np.empty(size, 'double')
    out = out_np
    
    for i in range(size):
        out[i] = inp[i]*inp[i]
    return out_np

In [ ]:
with nogil:
    for i in prange(size):
        out[i] = inp[i]*inp[i]

In [ ]:
for i in prange(size, nogil=True):
    out[i] = inp[i]*inp[i]

In [ ]:
for i in prange(size, nogil=True):
    out[i] = inp[i]*inp[i]
    with gil:
        x = 0 # Python assignment

In [ ]:
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize
hello_parallel = Extension('hello_parallel',
['hello_parallel.pyx'],
extra_compile_args=['-fopenmp'],
extra_link_args=['-fopenmp'])
setup(
name='Hello',
ext_modules = cythonize(['cevolve.pyx', hello_parallel]),
)

In [ ]:
def c_evolve(double[:, :] r_i,double[:] ang_speed_i,
    double timestep,int nsteps):
# cdef declarations
for i in range(nsteps):
    for j in range(nparticles):
        # loop body

In [ ]:
for j in range(nparticles):
    for i in range(nsteps):
# loop body

In [ ]:
for j in prange(nparticles, nogil=True)

In [ ]:
%timeit benchmark(10000, 'openmp') # Running on 4 processors

In [ ]:
%timeit benchmark(10000, 'cython')

## 7.4 Automatic parallelism

**Theano** is a project that allows you to define a mathematical expression on arrays (more generally, tensors), and compile them to a fast language, such as C or C++. Many of the operations that Theano implements are parallelizable and can run on both CPU and GPU.

**Tensorflow** is another library that, similar to Theano, is targeted towards expression of array-intensive mathematical expression but, rather than translating the expressions to specialized C code, executes the operations on an efficient C++ engine

Both Theano and Tensorflow are ideal when the problem at hand can be expressed in a chain of matrix and element-wise operations (such as neural networks).

### 7.4.1 Getting started with Theano

Theano is somewhat similar to a compiler but with the added bonuses of being able to express, manipulate, and optimize mathematical expressions as well as run code on CPU and GPU.

In [ ]:
conda install theano

In [ ]:
import theano.tensor as T
import theano as th
a = T.scalar('a')
a_sq = a ** 2
print(a_sq)
# Output:
# Elemwise{pow,no_inplace}.0

In [ ]:
compute_square = th.function([a], a_sq)

In [ ]:
compute_square(2)

In [ ]:
a.dtype

In [ ]:
a = T.vector('a')
b = T.vector('b')
ab_sq = a**2 + b**2
compute_square = th.function([a, b], ab_sq)
compute_square([0, 1, 2], [3, 4, 5])

In [ ]:
x = T.vector('x')
y = T.vector('y')
hit_test = x ** 2 + y ** 2 < 1

In [ ]:
hits = hit_test.sum()
total = x.shape[0]
pi_est = 4 * hits/total

In [ ]:
calculate_pi = th.function([x, y], pi_est)
x_val = np.random.uniform(-1, 1, 30000)
y_val = np.random.uniform(-1, 1, 30000)
import timeit
res = timeit.timeit("calculate_pi(x_val, y_val)",
"from __main__ import x_val, y_val, calculate_pi", number=100000)
print(res)
# Output:
# 10.905971487998613

In [ ]:
import theano
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 10

#### Profiling theano

### 7.4.2 Tensorflow

Tensorflow works by building mathematical expressions similar to Theano, except that the computation is not compiled to machine code but is executed on an external engine written in C++. Tensorflow supports execution and deployment of parallel codes on one or more CPUs and GPUs.

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
a = tf.placeholder('float64')

Tensorflow doesn't compile functions to C and then machine code like Theano, but serializes the defined mathematical functions (the data structure containing variables and transformations is called computation graph) and executes them on specific devices. The configuration of devices and context can be done using the tf.Session object.

In [ ]:
a = tf.placeholder('float64')
b = tf.placeholder('float64')
ab_sq = a**2 + b**2
with tf.Session() as session:
    result = session.run(ab_sq, feed_dict={a: [0, 1, 2],
                                            b: [3, 4, 5]})
print(result)
# Output:
# array([ 9., 17., 29.])

In [25]:
!python test_tensorflow.py 1

INTEL MKL ERROR: dlopen(/Users/boyuan/anaconda3/envs/Python/lib/libmkl_intel_thread.dylib, 9): Library not loaded: @rpath/libiomp5.dylib
  Referenced from: /Users/boyuan/anaconda3/envs/Python/lib/libmkl_intel_thread.dylib
  Reason: image not found.
Intel MKL FATAL ERROR: Cannot load libmkl_intel_thread.dylib.


The main advantage of using software packages such as Tensorflow and Theano is the support for parallel matrix operations that are commonly used in machine learning algorithms. This is very effective because those operations can achieve impressive performance gains on GPU hardware that is designed to perform these operations with high throughput.

### 7.4.3 Running code on a GPU

In [26]:
!THEANO_FLAGS=device=gpu python test_theano_gpu.py

python: can't open file 'test_theano_gpu.py': [Errno 2] No such file or directory


In [27]:
!THEANO_FLAGS=device=cpu python test_theano.py

INTEL MKL ERROR: dlopen(/Users/boyuan/anaconda3/envs/Python/lib/libmkl_intel_thread.dylib, 9): Library not loaded: @rpath/libiomp5.dylib
  Referenced from: /Users/boyuan/anaconda3/envs/Python/lib/libmkl_intel_thread.dylib
  Reason: image not found.
Intel MKL FATAL ERROR: Cannot load libmkl_intel_thread.dylib.


## 7.5 Summary